## Packages

In [6]:
import pandas as pd

from __future__ import print_function
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

## Constants

In [7]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SAMPLE_SPREADSHEET_ID = '1EHdPEQm9Suj9UZJd4n8vACkXLdzx4cQEun4Png__hTI'
SHEETS_NAMES = ['Buts', 'Dividendes', 'Dividendes Dernière journée', 'PPF', 'Passe Dé', 'Penalty Reussi',
                'CARTONS JAUNE', 'ARRETS GARDIEN', 'TITULARISATION', 'MATCH JOUÉ', 'PRIX', 'FBREF']

## Functions

In [8]:
def get_authenticated_service():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=8080)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('sheets', 'v4', credentials=creds)


def get_all_sheet_data(service):
    all_data = {}
    sheet = service.spreadsheets()

    for sheet_name in SHEETS_NAMES:
        range_name = f'{sheet_name}!A1:AJ500'  # Construct the range name
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=range_name).execute()
        values = result.get('values', [])

        if not values:
            continue

        all_data[sheet_name] = values

    return all_data


## SmokeTest

In [9]:
service = get_authenticated_service()
df = get_all_sheet_data(service)

In [10]:
fbref_df = pd.DataFrame(df.get('FBREF'))

#set column names equal to values in row index position 0
fbref_df.columns = fbref_df.iloc[0]
#remove first row from DataFrame
fbref_df = fbref_df[1:]

fbref_df

,Clt,NOM,Nation,Pos,Équipe,Âge,Naissance,MJ,Titulaire,Min,...,Buts/90,PD/90,B+PD/90,B-PénM/90,B+PD-PénM/90,xG/90,xAG/90,xG+xAG/90,npxG/90,npxG+xAG/90
1,1,Yunis Abdelhamid,ma MAR,DF,Reims,35-326,1987,1,1,90,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2,Salis Abdul Samed,gh GHA,MT,Lens,23-147,2000,1,1,90,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,3,Laurent Abergel,fr FRA,MT,Lorient,30-200,1993,1,1,90,...,0.00,0.00,0.00,0.00,0.00,0.06,0.01,0.07,0.06,0.07
4,4,Zakaria Aboukhlal,ma MAR,"MT,AT",Toulouse,23-183,2000,2,2,171,...,01.05,0.00,01.05,0.53,0.53,0.86,0.06,0.92,0.45,0.51
5,5,Akor Adams,ng NGA,AT,Montpellier,23-203,2000,2,2,166,...,1.63,0.00,1.63,1.63,1.63,0.87,0.19,01.06,0.87,01.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,286,Darlin Yongwa,cm CMR,DF,Lorient,22-332,2000,1,0,12,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
287,287,Warren Zaire-Emery,fr FRA,MT,Paris S-G,17-165,2006,2,2,180,...,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.02,0.00,0.02
288,288,Mehdi Zeffane,dz ALG,DF,Clermont Foot,31-093,1992,1,1,80,...,0.00,0.00,0.00,0.00,0.00,0.40,0.09,0.48,0.40,0.48
289,289,Nathan Zeze,fr FRA,DF,Nantes,18-063,2005,1,1,90,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
